# TRIAL NOTEBOOK

## Import Libraries

In [1]:
import os
import time
import pandas as pd
import nltk
import numpy as np

## Download NLTK Resources

In [2]:
# !mkdir -p ~/nltk_data
# !git clone https://github.com/nltk/nltk_data.git
# !mv nltk_data/packages/* ~/nltk_data/
# !rm -rf nltk_data

# Unzip punkt
# !WORK_DIR=$(pwd)
# !cd ~/nltk_data/tokenizers/
# !unzip punkt.zip
# !rm punkt.zip

# !cd $WORK_DIR

## Import helper functions

In [3]:
from utils import load_sessions, read_session, read_file
from operations import build_text

## Read File and Load Events

In [4]:
events = read_file("7c1e066d0c9c4901921cab117a0b7e73.jsonl")

Successfully read 1504 events in a writing session from ./coauthor-v1.0/7c1e066d0c9c4901921cab117a0b7e73.jsonl


## Build Text Buffer

In [5]:
text_buffer = []

In [6]:
for event in events:
    buffer = build_text(text_buffer, event)
    text_buffer.append(buffer)

## Initialize DataFrame

In [7]:
df = pd.DataFrame()

In [8]:
df["text_buffer"] = text_buffer
df["events"] = events

In [9]:
df.head()

,text_buffer,events
0,"A woman has been dating guy after guy, but it ...","{'eventName': 'system-initialize', 'eventSourc..."
1,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u..."
2,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u..."
3,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-get', 'eventSource':..."
4,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-open', 'eventSource'..."


## Extract Sentences

In [10]:
# Add sentences

sentence_buffer = []
num_sentences = []

for text in df["text_buffer"]:
    sentences = nltk.tokenize.sent_tokenize(text)
    sentence_buffer.append(sentences)
    num_sentences.append(len(sentences))

In [11]:
df["sentences"] = sentence_buffer
df["num_sentences"] = num_sentences

In [12]:
df.head()

,text_buffer,events,sentences,num_sentences
0,"A woman has been dating guy after guy, but it ...","{'eventName': 'system-initialize', 'eventSourc...","[A woman has been dating guy after guy, but it...",2
1,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u...","[A woman has been dating guy after guy, but it...",2
2,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u...","[A woman has been dating guy after guy, but it...",2
3,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-get', 'eventSource':...","[A woman has been dating guy after guy, but it...",2
4,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-open', 'eventSource'...","[A woman has been dating guy after guy, but it...",2


## Extract Event Names

In [13]:
df["event_name"] = df["events"].apply(lambda x: x["eventName"])

In [14]:
df.head()

,text_buffer,events,sentences,num_sentences,event_name
0,"A woman has been dating guy after guy, but it ...","{'eventName': 'system-initialize', 'eventSourc...","[A woman has been dating guy after guy, but it...",2,system-initialize
1,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u...","[A woman has been dating guy after guy, but it...",2,text-insert
2,"A woman has been dating guy after guy, but it ...","{'eventName': 'text-insert', 'eventSource': 'u...","[A woman has been dating guy after guy, but it...",2,text-insert
3,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-get', 'eventSource':...","[A woman has been dating guy after guy, but it...",2,suggestion-get
4,"A woman has been dating guy after guy, but it ...","{'eventName': 'suggestion-open', 'eventSource'...","[A woman has been dating guy after guy, but it...",2,suggestion-open


## Correct Number of Sentences in accordance to Event Name

In [15]:
df = df.groupby("num_sentences", group_keys=True).apply(lambda x: x)
df = df.sort_index()

In [16]:
df

text_buffer  \
num_sentences                                                           
2             0     A woman has been dating guy after guy, but it ...   
              1     A woman has been dating guy after guy, but it ...   
              2     A woman has been dating guy after guy, but it ...   
              3     A woman has been dating guy after guy, but it ...   
              4     A woman has been dating guy after guy, but it ...   
...                                                               ...   
45            1307  A woman has been dating guy after guy, but it ...   
              1308  A woman has been dating guy after guy, but it ...   
              1309  A woman has been dating guy after guy, but it ...   
              1496  A woman has been dating guy after guy, but it ...   
              1503  A woman has been dating guy after guy, but it ...   

                                                               events  \
num_sentences                                                           
2             0     {'eventName': 'system-initialize', 'eventSourc...   
              1     {'eventName': 'text-insert', 'eventSource': 'u...   
              2     {'eventName': 'text-insert', 'eventSource': 'u...   
              3     {'eventName': 'suggestion-get', 'eventSource':...   
              4     {'eventName': 'suggestion-open', 'eventSource'...   
...                                                               ...   
45            1307  {'eventName': 'text-insert', 'eventSource': 'u...   
              1308  {'eventName': 'text-insert', 'eventSource': 'u...   
              1309  {'eventName': 'cursor-select', 'eventSource': ...   
              1496  {'eventName': 'text-insert', 'eventSource': 'u...   
              1503  {'eventName': 'text-insert', 'eventSource': 'u...   

                                                            sentences  \
num_sentences                                                           
2             0     [A woman has been dating guy after guy, but it...   
              1     [A woman has been dating guy after guy, but it...   
              2     [A woman has been dating guy after guy, but it...   
              3     [A woman has been dating guy after guy, but it...   
              4     [A woman has been dating guy after guy, but it...   
...                                                               ...   
45            1307  [A woman has been dating guy after guy, but it...   
              1308  [A woman has been dating guy after guy, but it...   
              1309  [A woman has been dating guy after guy, but it...   
              1496  [A woman has been dating guy after guy, but it...   
              1503  [A woman has been dating guy after guy, but it...   

                    num_sentences         event_name  
num_sentences                                         
2             0                 2  system-initialize  
              1                 2        text-insert  
              2                 2        text-insert  
              3                 2     suggestion-get  
              4                 2    suggestion-open  
...                           ...                ...  
45            1307             45        text-insert  
              1308             45        text-insert  
              1309             45      cursor-select  
              1496             45        text-insert  
              1503             45        text-insert  

[1504 rows x 5 columns]

In [17]:
num_sentences = np.array(df["num_sentences"])
event_names = np.array(df["event_name"])

In [18]:
start_idx = 0
select_flag = False

for idx, event in enumerate(event_names):
    if event == "suggestion-get":
        start_idx = idx
    if event == "suggestion-select":
        select_flag = True
    if select_flag and event == "text-insert":
        if num_sentences[start_idx] == num_sentences[idx]:
            end_idx = idx + 1
        elif num_sentences[start_idx] < num_sentences[idx]:
            end_idx = idx
        for i in range(start_idx, end_idx):
            num_sentences[i] += 1
        select_flag = False

In [19]:
for idx, _ in enumerate(event_names):
    print(num_sentences[idx], event_names[idx])

2 system-initialize
2 text-insert
2 text-insert
2 suggestion-get
2 suggestion-open
2 suggestion-hover
2 suggestion-get
2 suggestion-close
2 suggestion-open
2 suggestion-hover
2 suggestion-get
2 suggestion-close
2 suggestion-open
2 suggestion-hover
2 suggestion-hover
2 suggestion-hover
2 suggestion-close
3 text-insert
3 text-insert
3 text-insert
3 text-delete
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
3 text-insert
4 suggestion-get
4 suggestion-open
4 suggestion-hover
4 suggestion-hover
4 suggestion-select
4 suggestion-close
4 text-insert
4 text-insert
5 suggestion-get
5 suggestion-open
5 suggestion-hover
5 suggestion-select
5 suggestion-close
5 text-insert
5 text-insert
6 suggestion-get
6 suggestio

In [27]:
df["num_sentences"] = num_sentences

## Compute Sentence and Event Sequence

In [21]:
np.unique(df["num_sentences"])

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [108]:
def compute_seq(events):
    
    # Remove suggestion-open, suggestion-hover
    events = np.delete(events, np.where(events == "suggestion-open"))
    events = np.delete(events, np.where(events == "suggestion-hover"))
    
    # Remove cursor-forward, cursor-backward, cursor-select
    events = np.delete(events, np.where(events == "cursor-forward"))
    events = np.delete(events, np.where(events == "cursor-backward"))
    events = np.delete(events, np.where(events == "cursor-select"))
    
    # Remove text-insert after suggestion-select
    select_flag = False
    new_events = []
    for idx, event in enumerate(events):
        if event == "suggestion-select":
            select_flag = True
        if event == "text-insert" and select_flag:
            select_flag = False
            continue
        new_events.append(event)
    events = np.array(new_events)
    
    # Remove text-delete
    events = np.delete(events, np.where(events == "text-delete"))
    
    # Remove suggestion-close
    events = np.delete(events, np.where(events == "suggestion-close"))
    
    # Identify GTP-3 calls
    events = events.tolist()
    start_idx = 0
    api_flag = False
    pop_idx = []
    for idx, event in enumerate(events):
        if event == "suggestion-get":
            start_idx = idx
            api_flag = True
        if event == "suggestion-select" and api_flag:
            api_flag = False
            for i in range(start_idx, idx):
                pop_idx.append(i)
    events = np.array(events)
    events = np.delete(events, pop_idx)
    
    # Group together text-inserts
    new_events = []
    temp = []
    for event in events:
        if event == "text-insert":
            temp.append(event)
        else:
            if len(temp) != 0:
                new_events.append("text-insert")
            new_events.append(event)
            temp = []
    if len(temp) != 0:
        new_events.append("text-insert")
    events = np.array(new_events)
    
    # Rename sequences
    seq_name_dict = {
        "system-initialize": "prompt",
        "text-insert": "user",
        "suggestion-get": "empty-call",
        "suggestion-select": "gpt3-call",
    }
    new_events = [seq_name_dict[event] for event in events]
    events = np.array(new_events)
    
    return events

In [111]:
for num in np.unique(df["num_sentences"]):
    sent = np.array(df[df["num_sentences"] == num]["text_buffer"])[-1]
    event_seq = np.array(df[df["num_sentences"] == num]["event_name"])
    print(num, compute_seq(event_seq))

2 ['prompt' 'user' 'empty-call' 'empty-call' 'empty-call']
3 ['user']
4 ['gpt3-call' 'user']
5 ['gpt3-call' 'user']
6 ['gpt3-call' 'user']
7 ['gpt3-call']
8 ['gpt3-call']
9 ['gpt3-call' 'user']
10 ['gpt3-call' 'user' 'empty-call' 'empty-call']
11 ['user']
12 ['gpt3-call' 'user']
13 ['gpt3-call' 'gpt3-call' 'user']
14 ['user']
15 ['gpt3-call' 'user']
16 ['gpt3-call' 'user']
17 ['gpt3-call']
18 ['gpt3-call' 'user' 'empty-call' 'user']
19 ['gpt3-call' 'user']
20 ['user']
21 ['user']
22 ['gpt3-call']
23 ['gpt3-call' 'user' 'empty-call' 'user']
24 ['user']
25 ['gpt3-call' 'user']
26 ['gpt3-call' 'user']
27 ['gpt3-call' 'user']
28 ['gpt3-call' 'user']
29 ['gpt3-call']
30 ['gpt3-call' 'user']
31 ['gpt3-call' 'empty-call' 'user']
32 ['gpt3-call' 'user']
33 ['gpt3-call']
34 ['user']
35 ['gpt3-call' 'user']
36 ['user' 'empty-call' 'user']
37 ['gpt3-call' 'user']
38 ['user' 'empty-call' 'user']
39 ['gpt3-call' 'user']
40 ['user']
41 ['gpt3-call' 'user' 'empty-call' 'user']
42 ['user' 'empty-call'

## Generate Graph